# Two period model tutorial

In this tutorial, we demonstrate the use of the dcegm package using the example of a simple consumption-retirement model which consists of two periods. Due to the simplicity of the considered model, it is well suited to become familiar with this kind of models and the use of the dcegm package.

We will start with the theoretical model and show how this model can be solved numerically using the dcegm package. Subsequently, we will derive the analytical solution of this problem and eventually, compare the two solutions in order to show the accuracy of the numerical solution by the DC-EGM algorithm.

### Two period model

We consider a simple consumption-retirement model with only two periods. The objective function in period 0 is given by

$$ V_0 = \max_{c_0, d_0} \mathbb{E}_\pi \left[ \sum_{t=0}^{1} \beta^{t} u(c_t,d_t)\right],$$

where $\beta \in [0,1]$ is the discount factor, $c_t\geq 0$ is the consumption in period $t$ and $d_t\in \{0,1\}$ specifies the choice with $d_t = 0$ indicating work and $d_t=1$ indicating retirement at the end of period $t$.
Here the utility function has the form

$$u(c_t,d_t) = \frac{c_t^{1-\rho}}{1-\rho}-\delta (1-d_t)+\epsilon_t(d_t)$$

The parameters $\rho \geq 0$ and $\delta \geq 0$ are measures of risk aversion and  disutility of work, respectively, while $\epsilon \sim EV(0,1)$ is a choice-specific taste shock with extreme-value distribution.


In each period $t$, the consumption $c_t$ has to satisfy the budget constraint $c_t \leq M_t$ with wealth

$$M_t = R(M_{t-1}-c_{t-1})+W_t(1-d_t)-K D_t,$$

where $R$ is the interest factor, $W_t$ is the wage in period $t$ and $D_t$ is an exogenous process indicating long-term care dependence with cost $K$.
The wage $W_t = W+\nu_t$ consists of an average wage $W$, which in the two-period model does not depend on $t$ as only the wage in period 1 enters the period 2 budget constraint, and an income shock $\nu_t \sim EV(0,1)$.
For the long-term care dependence, we have the following transition probabilities

\begin{equation*}
\pi(D_t=1\mid D_{t-1}=0)=p_t\\
\pi(D_t=1\mid D_{t-1}=1)=1
\end{equation*}

meaning that care dependence (just as retirement) is absorbing.

In [1]:
import jax.numpy as jnp
import numpy as np
from dcegm.solve import get_solve_func_for_model
from dcegm.pre_processing.setup_model import setup_model
from dcegm.interface import policy_and_value_for_state_choice_vec
from scipy.special import roots_sh_legendre
from scipy.stats import norm

from typing import Tuple
import jax

jax.config.update("jax_enable_x64", True)

### Solve model using DCEGM

We now demonstrate how to solve the model using the dcegm package. First, we define the parameters of the model. We store the parameters in a multi-index data frame. Inside the package the data frame is transformed into a dictionary with keys being the names of the index level `name`. They can be accessed in the functions defined by the user through the params_dict argument, which is one of the standard arguments of the functions. We will explain the set of keywords for each user function below.

In [2]:
params = {}
params["interest_rate"] = 0.02
params["ltc_cost"] = 5
params["max_wealth"] = 50
params["wage_avg"] = 8
params["sigma"] = 1
params["lambda"] = 1
params["ltc_prob"] = 0.3
params["beta"] = 0.95
params["rho"] = 0.9
params["delta"] = 1.5

Additionally, a dictionary containing at least the following options has to be specified:
- number of periods,
- number of discrete choices,
- size of the exogenous wealth grid,
- number of stochastic quadrature points,
- number of exogenous processes.

Furthermore, the user can specify arguments, which needs to use in the functions. These are expected to be constant independent of the parametrization of the model.

In [3]:
def prob_exog_ltc(ltc, params):
    prob_ltc = params["ltc_prob"]

    ltc_depenent = ltc == 1
    prob_ltc = ltc_depenent + (1 - ltc_depenent) * prob_ltc

    return jnp.array([1 - prob_ltc, prob_ltc])

In [4]:
options = {
    "model_params": {
        "quadrature_points_stochastic": 5,
        "n_choices": 2,
    },
    "state_space": {
        "n_periods": 2,
        "choices": np.arange(2),
        # "endogenous_states": {
        #     "married": [0, 1],
        # },
        "continuous_states": {
            "wealth": np.linspace(0, 50, 100),
        },
        "exogenous_processes": {
            "ltc": {"transition": prob_exog_ltc, "states": [0, 1]},
        },
    },
}

Beside the data frame ```params``` and the dictionary ```options```, the main function of the dcegm package ```solve_dcegm``` requires the following inputs:
- utility_functions,
- budget_constraint,
- solve_final_period,
- state_space_functions,
- user_transition_function.

In the following, we will explain the form of the required functions. Note that they all have to be JAX compatible, i.e. pure functions without if-conditions etc.

#### Utility functions

First, we define the utility, the marginal utility and inverse marginal utility. These functions are stored in a dictionary ```utility_functions```.

In [5]:
def flow_util(consumption, choice, params):
    rho = params["rho"]
    delta = params["delta"]
    u = consumption ** (1 - rho) / (1 - rho) - delta * (1 - choice)
    return u


def marginal_utility(consumption, params):
    rho = params["rho"]
    u_prime = consumption ** (-rho)
    return u_prime


def inverse_marginal_utility(marginal_utility, params):
    rho = params["rho"]
    return marginal_utility ** (-1 / rho)


utility_functions = {
    "utility": flow_util,
    "inverse_marginal_utility": inverse_marginal_utility,
    "marginal_utility": marginal_utility,
}

#### State space functions

Next we define state space functions ```create_state_space``` and ```state_specific_choice_set```. They can be directly imported the dcegm package, but we display them here, in order to explain how they work.

The function ```create_state_space``` depends only on the ```options``` dictionary and generates the state space, which is the collection of all states. A state consists of period, lagged choice and values of the exogenous processes.
Furthermore the function specifies an indexer object, which maps states to indexes.

The function ```get_specific_choice_set``` specifies possible choices for each state. Hence it takes into account the fact that retirement is absorbing, i.e. if $d_{t-1}=1$, then it must also hold that $d_{t}=1$.

In [6]:
def state_specific_choice_set(
    lagged_choice: np.ndarray,
    options: np.ndarray,
) -> np.ndarray:
    """Select state-specific choice set.

    Args:
        state (np.ndarray): Array of shape (n_state_variables,) defining the agent's
            state. In Ishkakov, an agent's state is defined by her (i) age (i.e. the
            current period) and (ii) her lagged labor market choice.
            Hence n_state_variables = 2.
        state_space (np.ndarray): Collection of all possible states of shape
            (n_periods * n_choices, n_choices).
        indexer (np.ndarray): Indexer object that maps states to indexes.
            Shape (n_periods, n_choices).

    Returns:
        choice_set (np.ndarray): The agent's (restricted) choice set in the given
            state of shape (n_admissible_choices,).

    """

    # Once the agent choses retirement, she can only choose retirement thereafter.
    # Hence, retirement is an absorbing state.
    if lagged_choice == 1:
        choice_set = np.array([1])
    else:
        choice_set = np.arange(options["n_choices"])

    return choice_set

As an example, we consider the 5th state of the state space, which corresponds to the case $d_1 = 0$, $D_1=0$. Consequently, in the second period, both choices $d_2 = 0$, $d_2 = 1$ are possible.

In [7]:
choice_set_5 = state_specific_choice_set(0, options["model_params"])
choice_set_5

array([0, 1])

Both function ```create_state_space``` and ```get_specific_choice_set``` are stored in a dictionary ```state_space_functions``` before being passed to the main function ```solve_dcegm```.

In [8]:
state_space_functions = {
    "state_specific_choice_set": state_specific_choice_set,
}

#### Budget function and transitions function

Moreover, we define the budget function as well as the transition function. Also note that these function work on the dictionary ```params_dict``` instead of the data frame ```params```.

In [9]:
def budget_dcegm(
    lagged_choice,
    ltc,
    savings_end_of_previous_period,
    income_shock_previous_period,
    params,
    options,
):  # noqa: 100
    interest_factor = 1 + params["interest_rate"]
    health_costs = params["ltc_cost"]
    wage = params["wage_avg"]
    resource = (
        interest_factor * savings_end_of_previous_period
        + (wage + income_shock_previous_period) * (1 - lagged_choice)
        - ltc * health_costs
    )
    return jnp.maximum(resource, 0.5)

#### Solve final period function

Lastly, a function that computes the solution to the final period is required. It can be imported directly from the package, but we also display it here.

In [10]:
def final_period_utility(wealth: float, choice: int, params) -> Tuple[float, float]:
    return flow_util(wealth, choice, params)


def marginal_final(wealth, choice):
    return marginal_utility(wealth, params)


final_period_utility = {
    "utility": final_period_utility,
    "marginal_utility": marginal_final,
}

#### Solve function

If all inputs have the required form as shown above, they can be passed to the function ```solve_dcegm```. This function returns two multi-dimensional arrays:

- policy (np.ndarray): Multi-dimensional np.ndarray storing the choice-specific policy function; of shape [n_states, n_discrete_choices, 2, 1.1 * n_grid_wealth]. Position $[.., 0, :]$ contains the endogenous grid over wealth M, and $[.., 1, :]$ stores the corresponding value of the policy function c(M, d), for each state and each discrete choice.
- value (np.ndarray): Multi-dimensional np.ndarray storing the choice-specific value functions; of shape [n_states, n_discrete_choices, 2, 1.1 * n_grid_wealth]. Position $[.., 0, :]$ contains the endogenous grid over wealth M, and $[.., 1, :]$ stores the corresponding value of the value function v(M, d), for each state and each discrete choice.


In [11]:
model = setup_model(
    options=options,
    utility_functions=utility_functions,
    utility_functions_final_period=final_period_utility,
    budget_constraint=budget_dcegm,
    state_space_functions=state_space_functions,
)

Update function for state space not given. Assume states only change with an increase of the period and lagged choice.
Sparsity condition not provided. Assume all states are valid.
Starting state space creation
State space created.

Starting state-choice space creation and child state mapping.
State, state-choice and child state mapping created.

Start creating batches for the model.
Model setup complete.



In [12]:
solve_func = get_solve_func_for_model(model)

In [13]:
value_solved, policy_solved, endog_grid_solved = solve_func(params)

### Analytical solution of the model

The solution of the given problem can be derived analytically using backwards induction.

#### Period 2: 
The choice problem in period 2 can be expressed through the Bellman equation
$$ V_2 = \max_{d_2\in \{0,1\}} \{v_2(M_2,d_2)+\epsilon_2(d_2)\}.$$
As this is the last period in our model and there is no bequest, the budget is consumed entirely , i.e. $c_2 = M_2$. Hence, the choice-specific value function for a given wealth level $M_2$ and choice $d_2$ is given by
$$ v_2(M_2,d_2) = \frac{M_2^{1-\rho}}{1-\rho} - \delta(1-d_2).$$

#### Period 1: 
Analogous to period 2, the choice problem in period 1 can be expressed through the Bellman equation


$$ V_1 = \max_{d_1\in \{0,1\}} \{v_1(M_1,d_1)+\epsilon_1(d_1)\}.$$


Here, the choice-specific value function for a given wealth level $M_1$ and choice $d_1$ is defined by

\begin{align*}
v_1(M_1,d_1) &= \max_{d_1\in\{0,1\}} \{u(c_1,d_1)+\beta E_1[EV_2(M_2(v_2,D_2))]\} \\
&= \max_{d_1\in\{0,1\}} \biggl\{u(c_1,d_1)+\beta \sum_{i=1}^{2}\left(\int EV_2(M_2(v_2,D_2))\, \text{d}f(\nu)\right)\pi(D_2 = i\mid D_1)\biggl\},
\end{align*}

where $EV_2(M_2(\nu_2,D_2))$ is the expected value function for a given realization of the income shock $\nu_2$ and exogenous process $D_2$, i.e. it is the expected maximum of the different choice specific value functions in the second period. 

The extreme value distribution takes the following closed formulas for the expected value function and choice probabilities:

$$EV_2(M_2) = \text{ln}(\text{exp}(v_2(M_2,1))+\text{exp}(v_2(M_2,0))),$$

$$P(d_2\mid M_2) = \frac{\text{exp}(v_2(M_2,d_2))}{\text{exp}(v_2(M_2,0))+\text{exp}(v_2(M_2,1))}.$$

Now the problem can be solved using the Euler equation (see Iskhakov et al, 2017, Appendix A, Lemma 1) given by

$$u^\prime(c_1\mid d_1) = \beta R E_1\left[\sum_{j=1}^{2} u^\prime(c_2(M_2\mid d_2),d_2) P(d_2 = j\mid M_2)\right]
.$$

#### Policy functions

Using the fact that the marginal utility is given by $u^\prime(c_t) = c_t^{-\rho}$ we obtain the consumption policy for period 1:

\begin{align*}
c_1 &= \left(\beta R E_1\left[\sum_{j=1}^{2} u^\prime(c_2(M_2\mid d_2),d_2) P(d_2 = j\mid M_2)\right]\right)^{-\frac{1}{\rho}} \\
&= \beta R  \sum_{i=1}^{2} \left( \int \sum_{j=1}^{2} u^\prime(c_2(M_2\mid d_2),d_2) P(d_2 = j\mid M_2)\, \text{d} f(\nu) \right)^{-\frac{1}{\rho}} \\
& = \beta R  \sum_{i=1}^{2} \left( \int \sum_{j=1}^{2} u^\prime(M_2,d_2) P(d_2 = j\mid M_2)\, \text{d} f(\nu) \right)^{-\frac{1}{\rho}},
\end{align*}

where we used the period-2 budget constraint $M_2 = c_2$ in the last equation. Note that this policy function is implicit, as $M_2$ depends on $c_1$. More specifically, we have 

$$M_2 = R(M_1-c_1)+W_2(1-d_1)-K D_2 = R(M_1-c_1)+(W+\nu_2)(1-d_1)-KD_2.$$

The labor supply in period 1 $d_1 \in \{0,1\}$ is the maximizer of $v_1(M_1,d_1)+\epsilon_1(d_1)$. Hence, $d_1 = 0$ if $v_1(M_1,0)+\epsilon_1(0)\geq v_1(M_1,1)+\epsilon_1(1)$ and $d_1 = 1$ otherwise.

Given $c_1$ and $d_1$, the consumption $c_2$ in period 2, which is equal to the wealth $M_2$ can be calculated by using the budget constraint. The labor supply $d_2$ in period 2 can be determined analogously to period 1 as a maximizer of $v_2(M_2,d_2)+\epsilon_2(d_2)$.

The budget constraint, wage, transition probability, choice probabilities and the right-hand side of the Euler equation can be implemented as follows.

In [14]:
def budget(
    lagged_resources, lagged_consumption, lagged_choice, wage, health, params_dict
):
    interest_factor = 1 + params_dict["interest_rate"]
    health_costs = params_dict["ltc_cost"]
    resources = (
        interest_factor * (lagged_resources - lagged_consumption)
        + wage * (1 - lagged_choice)
        - health * health_costs
    ).clip(min=0.5)
    return resources


def wage(nu, params_dict):
    wage = params_dict["wage_avg"] + nu
    return wage


def prob_long_term_care_patient(params_dict, lag_health, health):
    p = params_dict["ltc_prob"]
    if (lag_health == 0) and (health == 1):
        pi = p
    elif (lag_health == 0) and (health == 0):
        pi = 1 - p
    elif (lag_health == 1) and (health == 0):
        pi = 0
    elif (lag_health == 1) and (health == 1):
        pi = 1
    else:
        raise ValueError("Health state not defined.")

    return pi


def choice_probs(cons, d, params_dict):
    v = flow_util(cons, d, params_dict)
    v_0 = flow_util(cons, 0, params_dict)
    v_1 = flow_util(cons, 1, params_dict)
    choice_prob = np.exp(v) / (np.exp(v_0) + np.exp(v_1))
    return choice_prob


def m_util_aux(init_cond, params_dict, choice_1, nu, consumption):
    """Return the expected marginal utility for one realization of the wage shock."""
    budget_1 = init_cond["wealth"]
    health_state_1 = init_cond["health"]

    weighted_marginal = 0
    for health_state_2 in [0, 1]:
        for choice_2 in [0, 1]:
            budget_2 = budget(
                budget_1,
                consumption,
                choice_1,
                wage(nu, params_dict),
                health_state_2,
                params_dict,
            )
            marginal_util = marginal_utility(budget_2, params_dict)
            choice_prob = choice_probs(budget_2, choice_2, params_dict)
            health_prob = prob_long_term_care_patient(
                params_dict, health_state_1, health_state_2
            )
            weighted_marginal += choice_prob * health_prob * marginal_util

    return weighted_marginal


def euler_rhs(init_cond, params_dict, draws, weights, choice_1, consumption):
    beta = params_dict["beta"]
    interest_factor = 1 + params_dict["interest_rate"]

    rhs = 0
    for index_draw, draw in enumerate(draws):
        marg_util_draw = m_util_aux(init_cond, params_dict, choice_1, draw, consumption)
        rhs += weights[index_draw] * marg_util_draw
    return rhs * beta * interest_factor

### Comparison of DC-EGM algorithm and analytical solution

We now demonstrate the accuracy of the DC-EGM algorithm by inserting the calculated policy into the Euler equation and show that both sides take approximately the same value. 
As an example, let us consider a state in the state space with initial health $D_0 = 0$ and as initial wealth $M_0$, we take the first (non-zero) entry of the wealth grid. Furthermore, we choose $d_0 = 0$.

In [15]:
choice_in_period_0 = 0

# Prepare dictionary for closed form solution
initial_condition = {"health": 0, "wealth": 25}

In [16]:
state_dict = {
    "ltc": initial_condition["health"],
    "lagged_choice": 0,
    "period": 0,
    "choice": choice_in_period_0,
}

cons_calc, value = policy_and_value_for_state_choice_vec(
    endog_grid_solved=endog_grid_solved,
    value_solved=value_solved,
    policy_solved=policy_solved,
    params=params,
    model=model,
    state_choice_vec=state_dict,
    wealth=initial_condition["wealth"],
    compute_utility=model["model_funcs"]["compute_utility"],
)

Given the calculated policy using the dcegm package, we know compare the left hand side of the Euler equation (which is equal to the marginal utility) to its right hand side.

In [17]:
# needed for computation of the integral
quad_points, quad_weights = roots_sh_legendre(5)
quad_draws = norm.ppf(quad_points) * 1

As expected, the by the DC-EGM algorithm calculated consumption in the first period $c_1$ satisfies the Euler equation, since both sides are (approximately) equal.

In [18]:
rhs = euler_rhs(
    initial_condition, params, quad_draws, quad_weights, choice_in_period_0, cons_calc
)
rhs

Array(0.08292194, dtype=float64)

In [19]:
lhs = marginal_utility(cons_calc, params)
lhs

Array(0.08292197, dtype=float64)